## 뉴스 크롤링

In [ ]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from bs4.element import NavigableString

result = []
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36"}

for i in range(1,49) :
    #크롤링 할 링크 불러오기
    response = get("https://search.hankyung.com/apps.frm/search.news?query=%EC%A0%84%EA%B8%B0%EC%B0%A8&sort=DATE%2FDESC%2CRANK%2FDESC&period=DATE&area=title&mediaid_clust=HKPAPER&sdate=2020.11.01&edate=2021.04.30&exact=&include=%EC%A0%84%EA%B8%B0%EC%B0%A8&except=&page="+str(i),headers=headers)
    html = response.text
    if response.status_code == 200:
        bsObject = BeautifulSoup(html, 'html.parser')

        for newslist in bsObject.select('div.txt_wrap') :
            #한경 기사만 가져오기
            company = newslist.select_one('.info > span')
            if '한국경제' not in company : continue

            #제목
            newstitle = newslist.select_one('.tit').get_text().lstrip()
            if '[포토]' in newstitle : continue #포토 기사 제외
            if '전기차' not in newstitle : continue #제목에 '전기차'가 포함되어있지 않다면 제외

            #날짜/시간
            newsdate = newslist.select_one('span.date_time').string
            time = newsdate[11:] 
            newsdate = newsdate[:10]
            newsdate = newsdate[:4]+'년 '+newsdate[5:7]+'월 '+newsdate[8:]+'일'

            #본문
            url = newslist.a['href']
            response_news = get(url,headers=headers)
            newshtml = response_news.text

            if response_news.status_code == 200:
                bs = BeautifulSoup(newshtml, 'html.parser')
                content = bs.select_one("#articletxt")

                #기사 본문과 상관 없는 하위 태그들 삭제
                target_text_without_child_tags = [
                    bs_object
                    for bs_object
                    in content
                    if isinstance(bs_object, NavigableString)
                ]
                text = "".join(target_text_without_child_tags)
                content = text.strip()

                #본문에서 기자 삭제
                if '기사제보 및 보도자료 open@hankyung.com' in content :
                    content = content.replace('기사제보 및 보도자료 open@hankyung.com','')
                if '@hankyung.com' in content :
                    index = content.rfind('.',0,len(content)-13)
                    content = content[:index+1]

                #엑셀로 추출
                result.append([newstitle,newsdate,time,content])
                data = pd.DataFrame(result)
                data.to_csv('한경.csv', encoding='cp949') 
            else : 
                print(response_news.status_code)
    else :
        print(response.status_code)
    
    print('success {} page'.format(i))




## 주가 변동 도출

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
plt.style.use('seaborn')
import yfinance as yf 
from pykrx import stock

In [ ]:
hyundai = yf.download('005380.KS', start='2020-10-30', end='2021-04-30')
kia = yf.download('000270.KS', start='2020-10-30', end='2021-04-30')
lg_chemi = yf.download('051910.KS', start='2020-10-30', end='2021-04-30')
ss_sdi = yf.download('006400.KS', start='2020-10-30', end='2021-04-30')
sk_inov = yf.download('096770.KS', start='2020-10-30', end='2021-04-30')
posco_chemi = yf.download('003670.KS', start='2020-10-30', end='2021-04-30')
sk_net = yf.download('003670.KS', start='2020-10-30', end='2021-04-30')

In [ ]:
df = pd.DataFrame()
rdf = pd.DataFrame()

df['hyundai'] = hyundai.Close.pct_change().dropna()
rdf['hyundai'] = df.hyundai.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)


df['kia'] = kia.Close.pct_change().dropna()
rdf['kia'] = df.kia.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)


df['lg_chemi'] = lg_chemi.Close.pct_change().dropna()
rdf['lg_chemi'] = df.lg_chemi.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)


df['ss_sdi'] = ss_sdi.Close.pct_change().dropna()
rdf['ss_sdi'] = df.ss_sdi.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)


df['sk_inov'] = sk_inov.Close.pct_change().dropna()
rdf['sk_inov'] = df.sk_inov.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)


df['posco_chemi'] = posco_chemi.Close.pct_change().dropna()
rdf['posco_chemi'] = df.posco_chemi.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)


df['sk_net'] = sk_net.Close.pct_change().dropna()
rdf['sk_net'] = df.sk_net.apply(lambda x: [1, 0][x < 0])
df.dropna(inplace=True)
print(df.head())
print(rdf.head())

## 주가 변동 그래프 도출

In [ ]:
hyundai.Close.plot()
plt.savefig('./hyundai.png')

In [ ]:
kia.Close.plot()
plt.savefig('./kia.png')

In [ ]:
lg_chemi.Close.plot()
plt.savefig('./lg_chemi.png')

In [ ]:
lg_chemi.Close.plot()
plt.savefig('./lg_chemi.png')

In [ ]:
sk_inov.Close.plot()
plt.savefig('./sk_inov.png')

In [ ]:
posco_chemi.Close.plot()
plt.savefig('./posco_chemi.png')

In [ ]:
sk_net.Close.plot()
plt.savefig('./sk_net.png')

In [ ]:
df = pd.DataFrame() 
df['hyundai'] = hyundai.Close
df['kia'] = kia.Close
df['lg_chemi'] = lg_chemi.Close
df['ss_sdi'] = ss_sdi.Close
df['sk_inov'] = sk_inov.Close
df['posco_chemi'] = posco_chemi.Close
df['sk_net'] = sk_net.Close
df.plot()
plt.savefig('./kospi.png')

## 주가 종목별 뉴스 개수 추출

In [ ]:
#0으로 초기화
data = {'현대차':0,'기아차':0,'LG화학/에너지솔루션':0, '삼성SDI':0, 'SK이노베이션':0,'에코프로비엠':0, '엘앤에프':0, '포스코케미칼':0,'SK네트웍스':0, '피앤이솔루션':0, '피에스텍':0, '포스코ICT':0}

import pandas as pd

#뉴스 
data_pd = pd.read_csv(('total.csv'),encoding='cp949',header = None, index_col=None, names = None)
data_np = pd.DataFrame.to_numpy(data_pd)

#전기차 관련 주
i = 1
for i in range(1,2622) :
    title = data_np[i][1]
    if '현대' in title : data['현대차'] += 1
    if '기아' in title : data['기아차'] += 1
    if ('LG화학' or 'LG에너지솔루션') in title : data['LG화학/에너지솔루션'] += 1
    if '삼성SDI' in title : data['삼성SDI'] += 1
    if 'SK이노베이션' in title : data['SK이노베이션'] += 1
    if '에코프로비엠' in title : data['에코프로비엠'] += 1
    if '엘앤에프' in title : data['엘앤에프'] += 1
    if '포스코케미칼' in title : data['포스코케미칼'] += 1
    if 'SK네트웍스' in title : data['SK네트웍스'] += 1
    if '피앤이솔루션' in title : data['피앤이솔루션'] += 1
    if '피에스텍' in title : data['피에스텍'] += 1
    if (('(주)포스코 ICT') or ('POSCO ICT COMPANY LTD.') or ('포스데이타') or ('포스코아이씨티') or ('포스코 ICT') or ('포스코ICT')) in title : data['포스코ICT'] += 1
    i+=1

#데이터 출력
df = pd.DataFrame(data,index=['cnt'])
df = df.transpose()
df
